# Worthdale Vicinity Streets

This notebook details a quick analysis for Marketing to look at streets within 1/2 mile of Worthdale Park. The ideas is to use this list of streets to ensure that addresses nearest to the park get the mailer first.

In [ ]:
# Utilities Libraries
import requests
from io import BytesIO

# Data and Analysis
import pandas as pd
import geopandas as gpd

# Visualization Libraries
import contextily as ctx

## Helper Functions and Variables

In [ ]:
basemaps = {
    'streets': 'https://a.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}.png',
    'imagery': 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
}

## Data

For this project, we're going to need to pull data from a couple different datasets. We'll need the boundary for Worthdale Park and Wake County Streets. Fortunately, these are availabl via Raleigh Open Data and Wake County Open Data, respectively. Since this is a very localized analysis (1 park) and covers a small area (1/2 mile buffer of Worthdale Park) we probably don't need to work with all of Raleigh's parks boundaries nor Wake County streets. Fortunately, since these datasets are both available through ArcGIS REST Services, we can do some server-side filtering to pull in just the data we need to do our analysis around Worthdale Park.

As we move through this section, we'll load datasets and derive the 1/2 mile buffer for Worthdale Park. At the end, we'll have all the layers we need to generate our list of streets. As we go along, we'll plot some simple maps to see how all the layers fit together. 

### Worthdale Park Boundary

Creating URL query strings can be a fraught activity. Fortunately we can put all our query parameters in a dictionary and pass that with the base url for the Raleigh Parks REST query endpoint into `requests.get()`. This handy function will turn that base URL and dictionary into a well-formatted query and return the response. We can then use `BytesIO()` to decode the GeoJSON response and pass it to GeoPandas for conversion into a GeoDataFrame.

In [ ]:
parks_baseurl = 'https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/Parks/FeatureServer/0/query'
worthdale_params = {
    'f': 'geojson',
    'where': "NAME='Worthdale'",
}
worthdale_r = requests.get(parks_baseurl, params = worthdale_params)
worthdale_df = gpd.read_file(BytesIO(worthdale_r.content))
worthdale_df.head()

In [ ]:
ax1 = worthdale_df.to_crs(epsg = 3857).plot(alpha = 0.25, color = 'red', figsize = (18, 18))
ctx.add_basemap(ax1, url = basemaps['imagery'])

### Worthdale 1/2 Mile Buffer

We'll use geopandas to generate a 1/2 mile buffer around the Worthdale Park boundary. The approach below has us iterating through each row of `worthdale_df`. In truth, there is only one row, but I always forget that geopandas requires you to be a bit more explicit about which elements are being buffered. In other words, a hypothetical `all_parks_df.buffer(1000)` will not generate a 1000 unit buffer for each feature. You have to iterate through each row in your GeoDataFrame and apply the buffer one-by-one. It just requires a little different thinking than when using buffer tools in ArcGIS or QGIS. 

In [ ]:
worthdale_buffer_list = [[row['NAME'], row['geometry'].buffer(2640)] for idx, row in worthdale_df.iterrows()]
worthdale_buffer_df = gpd.GeoDataFrame(worthdale_buffer_list, columns = ['name', 'geometry'], geometry = 'geometry', crs = {'init': 'epsg:2264'})

In [ ]:
ax2 = worthdale_buffer_df.to_crs(epsg = 3857).plot(alpha = 0.25, color = 'yellow', figsize = (18, 18))
worthdale_df.to_crs(epsg = 3857).plot(ax = ax2, facecolor = (0,0,0,0), linestyle = '--', linewidth = 2, edgecolor = '#121212')
ctx.add_basemap(ax2, url = basemaps['streets'])

### Streets

There are a lot of streets in Wake County and we all need them for a small area. We can use the extent of `worthdale_buffer_df` in our request for the data so it only returns those streets that intersect that extent!

In [ ]:
streets_baseurl = 'https://maps.wakegov.com/arcgis/rest/services/Transportation/Transportation/MapServer/1/query'
worthdale_buffer_bounds = {}
worthdale_buffer_bounds['min_x'], worthdale_buffer_bounds['min_y'], worthdale_buffer_bounds['max_x'], worthdale_buffer_bounds['max_y'] = worthdale_buffer_df['geometry'].to_crs(epsg = 4326).iloc[0].bounds
streets_params = {
    'f': 'geojson',
    'where': "CLASSNAME NOT IN ('INT','RAMP')",
    'outFields': 'STNAME,CARTONAME',
    'geometryType': 'esriGeometryEnvelope',
    'geometry': f"{worthdale_buffer_bounds['min_x'], worthdale_buffer_bounds['min_y'], worthdale_buffer_bounds['max_x'], worthdale_buffer_bounds['max_y']}",
    'spatialRel': 'esriSpatialRelIntersects',
    'inSR': 4326,
    'outSR': 2264
}
streets_r = requests.get(streets_baseurl, params = streets_params)
streets_df = gpd.read_file(BytesIO(streets_r.content))

In [ ]:
ax3 = worthdale_buffer_df.to_crs(epsg = 3857).plot(alpha = 0.25, color = 'yellow', figsize = (18, 18))
streets_df.to_crs(epsg = 3857).plot(ax = ax3)
worthdale_df.to_crs(epsg = 3857).plot(ax = ax3, facecolor = (0,0,0,0), linestyle = '--', linewidth = 2, edgecolor = '#121212')
ctx.add_basemap(ax3, url = basemaps['streets'])

## Analysis

Now that we've got all our data gathered, let's work on generating that list of streets that are within that 1/2 mile buffer of Worthdale Park. 

In [ ]:
streets_halfmile_df = streets_df[streets_df['geometry'].intersects(worthdale_buffer_df.iloc[0]['geometry'])]
streets_halfmile_df.head()

In [ ]:
ax4 = worthdale_buffer_df.to_crs(epsg = 3857).plot(alpha = 0.15, color = 'yellow', figsize = (18, 18))
streets_halfmile_df.to_crs(epsg = 3857).plot(ax = ax4, alpha = 0.5, edgecolor = 'green', linewidth = 3)
worthdale_df.to_crs(epsg = 3857).plot(ax = ax4, facecolor = (0,0,0,0), linestyle = '--', linewidth = 2, edgecolor = '#121212')
ctx.add_basemap(ax4, url = basemaps['streets'])
ax4.axes.get_xaxis().set_visible(False)
ax4.axes.get_yaxis().set_visible(False)
ax4.axes.get_xaxis().set_ticks([])
ax4.axes.get_yaxis().set_ticks([])

The Marketing team really only needs a list of the unique street names that are within 1/2 mile of Worthdale Park. We can use the `groupby()` function in pandas to help generate a dataframe of unique street names only. While we're at it, we'll rename the columns to something that's more meaningful for humans.

In [ ]:
streets_halfmile_unique_df = streets_halfmile_df[['CARTONAME', 'STNAME']].groupby('CARTONAME').first().reset_index()
streets_halfmile_unique_df.columns = ['Full Street Name', 'Street Name']
streets_halfmile_unique_df.head()

We can check that there is indeed a difference between before and after running the `groupby()` function on `CARTONAME`.

In [ ]:
print(f'All values: {streets_halfmile_df["CARTONAME"].count()} | Unique values: {streets_halfmile_unique_df["Full Street Name"].count()}')

In case there was any more doubt, we can check all the values in the `Full Street Name` (nee `CARTONAME`) to make sure they're all unique.

In [ ]:
[s for s in streets_halfmile_unique_df['Full Street Name']]

Happy with the result, we can save the DataFrame of unique street names within 1/2 mile of Worthdale Park to a CSV for delivery to Marketing.

In [ ]:
streets_halfmile_unique_df.to_csv('worthdale_halfmile_streets.csv', index=False)

## Archive Data

Because we're pulling from live web-based services, there's a possibility that these data will either change or disappear. Let's archive the various data we produced so we have a snapshot of the data from when it was first analyzed on 8/15/2019.

In [ ]:
worthdale_df.to_crs(epsg = 4326).to_file('worthdale.geojson', driver = 'GeoJSON')
worthdale_buffer_df.to_crs(epsg = 4326).to_file('worthdale_halfmile_buffer.geojson', driver = 'GeoJSON')
streets_halfmile_df.to_crs(epsg = 4326).to_file('worthdale_halfmile_streets.geojson', driver = 'GeoJSON')